In [ ]:
from torch.utils.data import DataLoader
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

In [ ]:
from datasets import load_dataset
dataset = load_dataset("SetFit/sst2")

In [ ]:
import math
import datasets
import numpy as np
import random

id_to_char = ['<PAD>', '<CLS>', '<SEP>', '<MASK>', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', \
              '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']
char_to_id = {c:i for i,c in enumerate(id_to_char)}
chars = [' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', '[', '\\', ']', \
         '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']
rep = ['-lrb-', '-rrb-', 'ã§', 'ã¯', 'ã£', 'ã¨', 'ã»', 'ã¶', 'ã±', 'ã¢', 'ã-', 'ã¡', 'ã¦', 'ã³', 'ã©', 'ã¼', 'ü', 'û', 'ñ', 'ó', 'ô', 'ö', 'í', 'ï', 'mollã', 'jirã', 'ã', '\xad', '¼', '³', '¡', '¦', '\xa0', '¢', 'ç', '´', 'à', 'á', 'â', 'é', 'è', 'æ' ]
tok = ['('    , ')'    , 'c' , 'i' , 'a' , 'e' , 'u' , 'o' , 'n' , 'a' , 'i' , 'a' , 'ae', 'o' , 'e' , 'u' , 'u', 'u', 'n', 'o', 'o', 'o', 'i', 'i', 'molla', 'jiri', 'a', ''    , '' , '' , '' , '' , ''    , 'c', 'c', '' , 'a', 'a', 'a', 'e', 'e', 'ae']
assert(len(rep)==len(tok))

num_replace = 1
num_mod = 1
global_seed = 1

def gen_adv(examples):
    ret = []
    np.random.seed(global_seed)
    random.seed(global_seed)
    for text in examples['text']:
        text = refine_sentence(text)
        text = list(text)
        
        rpl = random.random() 
        ins = random.random()
        rmv = random.random()
        rpl = rpl/(rpl+ins+rmv)
        ins = ins/(rpl+ins+rmv)
        rpl = math.floor(num_mod*rpl)
        ins = math.floor(num_mod*ins)
        rmv = num_mod - rpl - ins
        
        #remove
        rnd = np.random.permutation(len(text))[:rmv]
        rnd *= -1
        rnd.sort()
        rnd *= -1
        for i in rnd:
            del text[i]
        
        #replace
        rnd = np.random.permutation(len(text))[:rpl]
        for i in rnd:
            text[i] = random.choice(chars)
        
        #insert
        rnd = np.random.permutation(len(text))[:ins]
        rnd *= -1
        rnd.sort()
        rnd *= -1
        for i in rnd:
            text.insert(i, random.choice(chars))
            
        ret.append(''.join(text))
    return {'text':ret}

# Deprecated
def replace_random(examples):
    ret = []
    np.random.seed(global_seed)
    random.seed(global_seed)
    for text in examples['text']:
        text = refine_sentence(text)
        text = list(text)
        rnd = np.random.permutation(len(text))
        for i in range(min(num_replace,len(rnd))):
            text[rnd[i]] = random.choice(chars)
        ret.append(''.join(text))
    return {'text':ret}

def refine_sentence(sent):
    sent = sent.lower()
    # Cut out \n|
    if sent[-1]=='\n': sent=sent[:-1]
    # Repleace LRB, RRB to (, ) respectively
    for f, t in zip(rep, tok):
        sent = sent.replace(f, t)
    return sent

def refine_dataset(texts):
    ret = []
    for text in texts:
        text = refine_sentence(text)
        ret.append(text)
    return ret

def preprocess(dataset):
    input_ids = [] # Ids for each character
    encoder_mask = [] # Mask for <PAD> tokens
    word_idx = [] # Indexes of ends of each word
    num_words = [] # Number of words
    num_chars = [] # Number of characters
    labels = []
    len_limit = 256
    
    for text, label in zip(dataset['text'], dataset['label']):
        d = [char_to_id['<CLS>']] # for input_ids
        w = [-1] # for word_idx, -1 for <CLS>
        n = 1 # for num_words
        for j, c in enumerate(text):
            d.append(char_to_id[c])
            if c==' ':
                w.append(0)
                n+=1
            else:
                w.append(n)

        w.append(n)
        len_d = len(d)-1 # Length except <CLS> and <SEP>
        d.append(char_to_id['<SEP>'])
        if len(d)<=len_limit: # Only add sentences with acceptable length
            mask = [0]*len(d) + [1]*(len_limit-len(d)) # Create mask
            d += [char_to_id['<PAD>']]*(len_limit-len(d)) # PAD current sentence
            w.append(-1) # <SEP>
            w += [-1]*(len_limit-len(w)) 

            # Append all to dataset
            input_ids.append(d)
            encoder_mask.append(mask)
            word_idx.append(w)
            num_words.append(n)
            num_chars.append(len_d)
            labels.append(label)

        # Reset variables to start a new sentence
        d = [char_to_id['<CLS>']]
        w = [-1]
        n = 1
        
    input_ids = [x for x in torch.tensor(input_ids)]
    encoder_mask = [x for x in torch.tensor(encoder_mask)]
    label = [x for x in torch.tensor(labels)]
    dataframe = list(zip(input_ids, encoder_mask, label))
    return DataLoader(dataframe, batch_size=48, shuffle=True, num_workers=4)

In [ ]:
from torch.nn import ModuleList
import copy

# Vocab
id_to_char = ['<PAD>', '<CLS>', '<SEP>', '<MASK>', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', \
              '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']
char_to_id = {c:i for i,c in enumerate(id_to_char)}
num_spechar = 4
vocab_size = len(id_to_char)-num_spechar
data_max_len = 256


class EmbeddingLayer(nn.Module):
    def __init__(self, vocab_size, embed_size, pad_idx, max_seq_len, drop_prob):
        super(EmbeddingLayer, self).__init__()
        self.max_seq_len = max_seq_len
        self.char_embedding = nn.Embedding(vocab_size, embed_size, padding_idx=pad_idx)
        self.position_embedding = nn.Embedding(max_seq_len, embed_size)
        self.LayerNorm = nn.LayerNorm(embed_size, eps=1e-7)
        self.dropout = nn.Dropout(drop_prob)  # 0.1


    def forward(self, input_ids):
        position_ids = torch.arange(self.max_seq_len, dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)

        words_embeddings = self.char_embedding(input_ids)
        position_embeddings = self.position_embedding(position_ids)
        
        embeddings = words_embeddings + position_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings  # (batchSize, sequenceLength, hidden_size)

class CBERT(nn.Module):
    def __init__(self, vocab_size, embed_size, dim_feedforward, num_heads, num_layers, pad_idx):
        super(CBERT, self).__init__()
        self.embedding_layer = EmbeddingLayer(vocab_size, embed_size, pad_idx, data_max_len, 0.1)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_size, nhead=num_heads, dim_feedforward=dim_feedforward, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.prediction_layer = nn.Linear(embed_size, vocab_size)
        self.log_softmax = nn.LogSoftmax(dim=2)
    
    def forward(self, x, mask):
        embedded_x = self.embedding_layer(x)
        encoded_x = self.transformer_encoder(embedded_x, src_key_padding_mask=mask)
        #return torch.flatten(encoded_x, start_dim=1)
        return encoded_x[:,0,:]
        #return torch.mean(encoded_x, 1, False)
    
class CBERT_SA(nn.Module):
    def __init__(self, vocab_size, embed_size, dim_feedforward, num_heads, num_layers, pad_idx, CBERT_PATH, num_class=2):
        super(CBERT_SA, self).__init__()
        self.CBERT = CBERT(vocab_size, embed_size, dim_feedforward, num_heads, num_layers, pad_idx)
        self.prediction_layer = nn.Linear(embed_size, num_class)
        #self.prediction_layer = nn.Sequential(nn.Linear(embed_size*256, num_class))
        #self.prediction_layer = nn.Sequential(nn.Linear(embed_size, num_class))
        #self.prediction_layer = nn.Sequential(nn.Linear(embed_size, 30), nn.ReLU(), nn.Linear(30, num_class))
        self.softmax = nn.Softmax(dim=1)
        
        # init
        checkpoint = torch.load(CBERT_PATH)
        self.CBERT.load_state_dict(checkpoint['model_state_dict'])

        def init_weights(m):
            if type(m) == nn.Linear:
                torch.nn.init.xavier_uniform_(m.weight)
                torch.nn.init.zeros_(m.bias)
        self.prediction_layer.apply(init_weights)
    
    def forward(self, x, mask):
        #print(x.shape)
        encoded_x = self.CBERT(x, mask)
        #print(encoded_x.shape)
        pred = self.prediction_layer(encoded_x)
        #print(pred.shape)
        ret = self.softmax(pred)
        #print(ret)
        return ret

In [ ]:
model = CBERT_SA(vocab_size=len(id_to_char),
                 embed_size=768,
                 dim_feedforward=2048,
                 num_heads=12,
                 num_layers=6,
                 pad_idx=char_to_id['<PAD>'],
                 CBERT_PATH = "./models/yelp4_cp12.pt")
model = model.to(device='cuda')

In [ ]:
LOAD_PATH = "./models/SST_cp0.pt"
checkpoint = torch.load(LOAD_PATH)
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
def evaluate(model, loss_f, val_iter, device='cpu'):
    model = model.to(device=device)
    model.eval()
    with torch.no_grad():
        loss_sum = torch.tensor([0.0], device=device)
        acc_sum = torch.tensor([0.0], device=device)
        n = 0
        
        for x, mask, y in tqdm(val_iter):
            x, mask, y = x.to(device=device), mask.to(device=device), y.to(device=device)
            y_hat = model(x, mask)            
            loss = loss_f(y_hat, y)
        
            loss_sum += loss.float()
            pred = torch.argmax(y_hat, dim=1)
            acc_sum += torch.sum(pred==y)
            n += x.shape[0]
    
    return (loss_sum/n, acc_sum/n)

    
def print_decoded_ids(ids):
    for c in ids:
        if c==0: # <PAD>, <SEP>
            break
        else:
            print(id_to_char[c], end='')
    print()    

def show_sample(model, loss_f, data_iter, device='cpu'):
    model = model.to(device=device)
    model.eval()
    with torch.no_grad():
        idx=0
        for x, mask, y in data_iter:
            x, mask, y = x.to(device=device), mask.to(device=device), y.to(device=device)
            model.train()
            y_hat = model(x, mask)            
            loss = loss_f(y_hat, y)

            pred = torch.argmax(y_hat, dim=1)
            for i,p,t in zip(x, pred, y):
                print("INPUT:", end=' ')
                print_decoded_ids(i)
                print("TARGET:", t.item(), end=' ')
                print("PREDICTION:", p.item())
            print("loss:", loss)
            pred = torch.argmax(y_hat, dim=1)
            print("acc:",torch.sum(pred==y)/x.shape[0])
            print("="*80)

            if idx==1: break
            else: idx+=1

In [ ]:
from tqdm import tqdm
loss_f = nn.CrossEntropyLoss()
seeds = [31, 41, 59, 2, 65, 35, 897, 93, 2384, 626]
#seeds = [31]
val_result = []
test_result = []
for n_r in range(11):
    print(n_r)
    num_mod = n_r
    val_rt = []
    test_rt = []
    for seed in seeds:
        global_seed = seed
        #replaced_dataset = dataset.map(replace_random, batched=True)
        replaced_dataset = dataset.map(gen_adv, batched=True)
        val_iter = preprocess(replaced_dataset['validation'])
        test_iter = preprocess(replaced_dataset['test'])
        val_r = evaluate(model, loss_f, val_iter, device='cuda')
        val_rt.append(val_r)
        test_r = evaluate(model, loss_f, test_iter, device='cuda')
        test_rt.append(test_r)
    val_result.append(val_rt)
    test_result.append(test_rt)

In [ ]:
for k in range(11):
    print(k)
    avg = 0.0
    for r in val_result[k]:
        print(r[1].item(), end=' ')
        avg += r[1].item()
    print("AVG: ", avg/len(val_result[k]))
print("="*80)
for k in range(11):
    print(k)
    avg = 0.0
    for r in test_result[k]:
        print(r[1].item(), end=' ')
        avg += r[1].item()
    print("AVG: ", avg/len(val_result[k]))